### Please install the required Python modules/SDKs

In [1]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

Could not find conda environment: ai-azure-c1
You can list all discoverable environments with `conda info --envs`.



# Azure Custom Vision - Image Classification Demo

## Importing utility functions and Python modules

In [2]:
from PIL import Image
import matplotlib.pyplot as plt

import os, time, uuid

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


### Resources:
- Azure Custom Vision Endpoint
- Training Reource ID and Key
- Prediction Resource ID and Key

## Make sure you have the correct Training and Prediction Endpoints, Keys and Resource IDs separately

In [4]:
TRAINING_ENDPOINT = "https://nayanacustomvision.cognitiveservices.azure.com/"
training_key = "ce08598caf404621b37136119dccd921"
training_resource_id = "/subscriptions/bb272072-9c6d-4e28-b814-947814c3e6ef/resourceGroups/nayana-ai-msft-azure/providers/Microsoft.CognitiveServices/accounts/nayanacustomvision"

In [5]:
PREDICTION_ENDPOINT = "https://nayanacustomvision-prediction.cognitiveservices.azure.com/"
prediction_key = "6b7c1fe2bea0431a8a3b4259749591e6"
prediction_resource_id = "/subscriptions/bb272072-9c6d-4e28-b814-947814c3e6ef/resourceGroups/nayana-ai-msft-azure/providers/Microsoft.CognitiveServices/accounts/nayanacustomvision-Prediction"

## Instantiate and authenticate the training client with endpoint and key 

In [6]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [7]:
trainer.api_version

'3.4-preview'

## Creating Training Project First

In [8]:
# Create a new project
print ("Training project created. Proceed to the next cell.")
project_name = uuid.uuid4()
project = trainer.create_project(project_name)

Training project created. Proceed to the next cell.


## Getting Project Details as collective information 

In [9]:
project.as_dict()

{'id': '5fcb2b44-ec8b-4b30-b430-b835afe8ec74',
 'name': 'e859911c-a2d8-473b-a27a-7497595f24ad',
 'description': '',
 'settings': {'domain_id': 'ee85a74c-405e-4adc-bb47-ffa8ca0c9f31',
  'classification_type': 'Multilabel',
  'target_export_platforms': [],
  'use_negative_set': True,
  'image_processing_settings': {'augmentation_methods': {'rotation': True,
    'scaling': True,
    'translation': True,
    'horizontal flip': True,
    'equalize': True,
    'solarize': True,
    'padtosquare': True}}},
 'created': '2022-06-15T18:21:31.643Z',
 'last_modified': '2022-06-15T18:21:31.643Z',
 'dr_mode_enabled': False,
 'status': 'Succeeded'}

## Adding Tags based on training requirements
- We have 3 tags in the training process 
  - nature
  - architecture
  - interior

In [10]:
nature_tag = trainer.create_tag(project.id, "Nature")

In [11]:
architecture_tag = trainer.create_tag(project.id, "Architecture")

In [12]:
interior_tag = trainer.create_tag(project.id, "Interior")

## Upload Traning Data 

### Enter local file system location of the traning images 
- All training images are saved in the file system within this workspace environment. 
- You can click on the "Jupyter" icon on the top left of this workspace to view these image folders:
    - You will find all nature images in the `nature-images` folder
    - You will find all architecture images in the `architecture-images` folder
    - You will find all the interior images in the `interior-images` folder
    - There are also three test images for you to perform predictions later.

In [13]:
# Get current working directory
# The output will give you the "local_image_path" used in the cell below
!pwd

/Users/nayanex/Desktop/Udacity/AI-Engineer-using-Microsoft-Azure-Nanodegree-Program/Building Computer Vision Solutions with Azure/Lesson 5 - Image Classification and Object Detection


In [14]:
local_image_path = ''

In [15]:
# Some code is taken from Azure SDK Sample and added my own code here
def upload_images_for_training(local_project_id, local_img_folder_name, image_tag_id):
    image_list = []
    files = os.listdir(os.path.join (local_image_path, local_img_folder_name))
    for file in files:
        full_path = os.path.join(local_image_path, local_img_folder_name, file)
        if os.path.isfile(full_path) and full_path.endswith('.jpeg'):
            with open(os.path.join (local_image_path, local_img_folder_name, file), "rb") as image_contents:
                image_list.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), tag_ids=[image_tag_id]))
                
    upload_result = trainer.create_images_from_files(local_project_id, ImageFileCreateBatch(images=image_list))
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
    return upload_result

In [16]:
nature_upload_result = upload_images_for_training(project.id, 'nature-images', nature_tag.id)

In [17]:
nature_upload_result.is_batch_successful

True

In [18]:
architecture_upload_result = upload_images_for_training(project.id, 'architecture-images', architecture_tag.id)

In [19]:
architecture_upload_result.is_batch_successful

True

In [20]:
interior_upload_result = upload_images_for_training(project.id, 'interior-images', interior_tag.id)

In [21]:
interior_upload_result.is_batch_successful

True

## Start the Image Classification Training
- We will keep checking the training progress every 10 seconds

In [22]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 sec

## After training is complete, let's look at the Model Performance

In [23]:
iteration.as_dict()

{'id': 'd8e23eaa-4fc1-4ba8-85cd-5b90a8f4e980',
 'name': 'Iteration 1',
 'status': 'Completed',
 'created': '2022-06-15T18:21:31.646Z',
 'last_modified': '2022-06-15T18:29:24.820Z',
 'trained_at': '2022-06-15T18:29:24.774Z',
 'project_id': '5fcb2b44-ec8b-4b30-b430-b835afe8ec74',
 'exportable': False,
 'domain_id': 'ee85a74c-405e-4adc-bb47-ffa8ca0c9f31',
 'classification_type': 'Multilabel',
 'training_type': 'Regular',
 'reserved_budget_in_hours': 0,
 'training_time_in_minutes': 4}

In [24]:
iteration_list = trainer.get_iterations(project.id)
for iteration_item in iteration_list:
    print(iteration_item)

{'additional_properties': {}, 'id': 'd8e23eaa-4fc1-4ba8-85cd-5b90a8f4e980', 'name': 'Iteration 1', 'status': 'Completed', 'created': datetime.datetime(2022, 6, 15, 18, 21, 31, 646000, tzinfo=<isodate.tzinfo.Utc object at 0x7fc06b47c940>), 'last_modified': datetime.datetime(2022, 6, 15, 18, 29, 24, 820000, tzinfo=<isodate.tzinfo.Utc object at 0x7fc06b47c940>), 'trained_at': datetime.datetime(2022, 6, 15, 18, 29, 24, 774000, tzinfo=<isodate.tzinfo.Utc object at 0x7fc06b47c940>), 'project_id': '5fcb2b44-ec8b-4b30-b430-b835afe8ec74', 'exportable': False, 'exportable_to': None, 'domain_id': 'ee85a74c-405e-4adc-bb47-ffa8ca0c9f31', 'classification_type': 'Multilabel', 'training_type': 'Regular', 'reserved_budget_in_hours': 0, 'training_time_in_minutes': 4, 'publish_name': None, 'original_publish_resource_id': None, 'custom_base_model_info': None, 'training_error_details': None}


In [25]:
model_perf = trainer.get_iteration_performance(project.id, iteration_list[0].id)

In [26]:
model_perf.as_dict()

{'per_tag_performance': [{'id': 'de75ded1-4837-418f-b9eb-8d23d5b82d0f',
   'name': 'Architecture',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 1.0,
   'recall_std_deviation': 0.0,
   'average_precision': 1.0},
  {'id': 'fde7f5d6-f3d1-4d9a-8612-bcd36bf69ea0',
   'name': 'Interior',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 1.0,
   'recall_std_deviation': 0.0,
   'average_precision': 1.0},
  {'id': '9a765e97-345c-4d36-8bc0-5d4bb8763abc',
   'name': 'Nature',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 1.0,
   'recall_std_deviation': 0.0,
   'average_precision': 1.0}],
 'precision': 1.0,
 'precision_std_deviation': 0.0,
 'recall': 1.0,
 'recall_std_deviation': 0.0,
 'average_precision': 1.0}

## Publishing the Model to the Project Endpoint

In [27]:
# Setting the Iteration Name, this will be used when Model training is completed
# Please choose a name favorable to you.
publish_iteration_name = "udacity-3-class-classification-demo-custom"

In [28]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Instantiate and authenticate the prediction client with endpoint and key

In [29]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [30]:
predictor.api_version

'3.1'

## Performing Prediction
- Using the predictor object 

In [31]:
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

## The test images are stored in the local file system of this workspace
* You will perform prediction twice below

In [32]:
# To list the folders/files in your current working directory
# The name of any image file can be used as "file_name" in the cell below
!ls

L5 Demo 1 - Predict API.ipynb          architecture-images
L5 Demo 2 - Image Classification.ipynb interior-images
L5 Image Classification Exercise.ipynb nature-images
README.md                              test-architecture
animals                                test-nature


In [35]:
file_name = 'test-architecture-01.jpeg'

In [36]:
# Pick one test image file name from the output of the previous cell
# Use the same image file name for this cell and the next one
perform_prediction(file_name)

FileNotFoundError: [Errno 2] No such file or directory: 'test-architecture-01.jpeg'

In [ ]:
# Checking the image
with open(os.path.join (local_image_path, file_name), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

In [ ]:
file_name_2 = 'test-nature-02.jpg'

In [ ]:
# Perform prediction again using another image
perform_prediction(file_name_2)

In [ ]:
# Checking the second image
with open(os.path.join (local_image_path, file_name_2), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)